<a href="https://colab.research.google.com/github/kevin6449/ironman2024_genai/blob/main/gen_ai_day14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 函式呼叫

函式呼叫可讓您輕鬆取得結構化資料輸出內容 生成式模型接著，您可以使用這些輸出內容呼叫其他 API，並將相關回應資料傳回模型。換句話說，函式呼叫有助於 必須連結生成式模型與外部系統 內含最新且準確的資訊。

In [8]:
import google.generativeai as genai

In [9]:
from google.colab import userdata

API_KEY=userdata.get('GOOGLE_API_KEY')

In [10]:
#genai.configure(api_key="YOUR_API_KEY")

# Configure the client library by providing your API key.
genai.configure(api_key=API_KEY)

### 平行函式呼叫

定義工具

In [11]:
def power_disco_ball(power: bool) -> bool:
    """供電旋轉的迪斯可球。"""
    print(f"迪斯可球 是 {'旋轉!' if power else '停止.'}")
    return True


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """播放一些符合指定參數的音樂。

    Args:
      energetic：音樂是否充滿活力。
      loud：音樂是否響亮。
      bpm：音樂每分鐘的節拍數。

    Returns: 正在播放的歌曲的名稱。
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "絕對不會放棄你。"


def dim_lights(brightness: float) -> bool:
    """調暗燈光.

    Args:
      brightness: 燈光的亮度，0.0為關閉，1.0為全亮。
    """
    print(f"燈光現已設定為 {brightness:.0%}")
    return True

### 可使用所有指定工具的指令來呼叫模型

In [12]:
# Set the model up with tools.
house_fns = [power_disco_ball, start_music, dim_lights]

model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=house_fns)

# Call the API.
chat = model.start_chat()
response = chat.send_message("把這個地方變成一個聚會！")

# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

power_disco_ball(power=True)
start_music(bpm=120.0, energetic=True, loud=True)
dim_lights(brightness=0.5)


每個顯示的結果都代表模型要求的單一函式呼叫。如要傳回結果，請按照要求的順序加入回應。

In [13]:
# Simulate the responses from the specified tools.
responses = {
    "power_disco_ball": True,
    "start_music": "Never gonna give you up.",
    "dim_lights": True,
}

# Build the response parts.
response_parts = [
    genai.protos.Part(function_response=genai.protos.FunctionResponse(name=fn, response={"result": val}))
    for fn, val in responses.items()
]

response = chat.send_message(response_parts)
print(response.text)

好的！我已經打開了迪斯可球、播放了活力十足的音樂，並將燈光調暗了一點。準備好派對吧！ 



#AllowedType = (int | float | bool | str | list['AllowedType'] | dict[str, AllowedType])

### 函式呼叫資料類型對應

描述您傳遞至模型的函式

In [14]:
def multiply(a:float, b:float):
    """returns a * b."""
    return a*b

model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                             tools=[multiply])

model._tools.to_proto()

[function_declarations {
   name: "multiply"
   description: "returns a * b."
   parameters {
     type_: OBJECT
     properties {
       key: "a"
       value {
         type_: NUMBER
       }
     }
     properties {
       key: "b"
       value {
         type_: NUMBER
       }
     }
     required: "a"
     required: "b"
   }
 }]

### 使用 genai.protos 類別編寫的相同乘法函式宣告

In [15]:
calculator = genai.protos.Tool(
    function_declarations=[
      genai.protos.FunctionDeclaration(
        name='multiply',
        description="傳回兩個數字的乘積。",
        parameters=genai.protos.Schema(
            type=genai.protos.Type.OBJECT,
            properties={
                'a':genai.protos.Schema(type=genai.protos.Type.NUMBER),
                'b':genai.protos.Schema(type=genai.protos.Type.NUMBER)
            },
            required=['a','b']
        )
      )
    ])

將此項目描述為與 JSON 相容的物件

In [16]:
calculator = {'function_declarations': [
      {'name': 'multiply',
       'description': '傳回兩個數字的乘積。',
       'parameters': {'type_': 'OBJECT',
       'properties': {
         'a': {'type_': 'NUMBER'},
         'b': {'type_': 'NUMBER'} },
       'required': ['a', 'b']} }]}

將 genai.protos.Tool 的表示法或工具清單傳遞

In [17]:
model = genai.GenerativeModel('gemini-1.5-flash', tools=calculator)
chat = model.start_chat()

response = chat.send_message(
    f"234551 X 325552 是多少？",
)
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "multiply",
                  "args": {
                    "b": 325552.0,
                    "a": 234551.0
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category

模型傳回 genai.protos.FunctionCall，叫用計算機的 multiply 函式

In [20]:
response.candidates

[content {
  parts {
    text: "234551 X 325552 \347\255\211\346\226\274 76358547152\343\200\202"
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
]

### 自行執行函式

In [18]:
fc = response.candidates[0].content.parts[0].function_call
assert fc.name == 'multiply'

result = fc.args['a'] * fc.args['b']
result

76358547152.0

將結果傳送至模型，即可繼續對話

In [19]:
response = chat.send_message(
    genai.protos.Content(
    parts=[genai.protos.Part(
        function_response = genai.protos.FunctionResponse(
          name='multiply',
          response={'result': result}))]))

print(response.text)

234551 X 325552 等於 76358547152。
